In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import seaborn as sns
#pd.set_option('display.min_rows', 6)
#pd.set_option('display.max_rows', 100)
#pd.set_option('display.max_columns', None)

#### 1 - Cargamos el dataset limpio

In [2]:
df = pd.read_csv('jugadores_titulares4.csv')

In [3]:
#fig_dims = (20, 16)
#plt.subplots(figsize=fig_dims)
#sns.heatmap(datos, annot=True)

In [4]:
#df.to_csv('jugadores_titulares4.csv', index=False)

In [3]:
#limpieza de strings,nans mover columna 'CF' al final
#df = df.drop(['Unnamed: 0','Eventos'],axis=1)

cf = df['CF']
df.drop(labels=['CF'], axis=1,inplace = True)
df.insert(10,'CF', cf)

In [6]:
#Cambiar jugadores a numeros
#keys = df['Titulares'].unique()
#values = range(len(keys))
#dicc_jug = dict(zip(keys,values))
#df['Titulares'] = df['Titulares'].replace(dicc_jug)
#df['Titulares'].value_counts()
#dicc_jug

In [4]:
list_equip = list(df['equipo'].unique())

In [5]:
df.head()

,Titulares,T,TaP,C,CP,Reg,PA,partido,equipo,Jornada,CF
0,0,0.0,0.0,0.0,0.0,0,0,1,girona,1,0.0
1,1,0.0,0.0,0.0,0.0,0,0,1,girona,1,0.0
2,2,0.0,0.0,0.0,0.0,0,1,1,girona,1,0.0
3,3,1.0,0.0,2.0,0.0,1,2,1,girona,1,0.0
4,4,0.0,0.0,6.0,1.0,2,0,1,girona,1,1.0


In [9]:
# Elegir el equipo a predecir

# partidos[partidos['Jornada']>=33]
#huesca 64

#### 2- Definimos la función para crear en formato vectorial un dataframe para cada equipo.
#### Esto nos permite pasar a un dataset con formato vectorial, donde cada línea está formada por los resultados de un equipo en sus 2 jornadas anteriores, de los datos conocidos de la jornada a jugarse y nuestra variable.
#### Este modelo es el que refleja más fielmente el rendimiento de los jugadores ya que si incluyésemos los datos de hace 10 jornadas (por ejemplo), la desviación sería muy elevada. O dicho en otras palabras, cuando un jugador juega una semana, lo más probable que la semana siguiente mantenga un nivel aproximado de juego, sin embargo, en 10 jornadas pueden pasar 2,5 o 3 meses, tiempo suficiente para que se alteren los niveles de juego y los datos arrojados.

In [6]:
def Team(E):
    partidos = df.groupby('equipo').get_group(E)
    x = 1 #jornada a empezar
    lista = []
    for i in range(36): #numero de filas
        datos_a = pd.DataFrame(np.concatenate(np.asarray(partidos[partidos.Jornada == i+x].iloc[0:12,0:7]))).T
        datos_a = datos_a.reset_index()
        corners_a = partidos[partidos.Jornada == i+x].iloc[0:12,-1].sum()#suma de los corners
        datos_a.insert(6,'corners', corners_a)


        datos_b = pd.DataFrame(np.concatenate(np.asarray(partidos[partidos.Jornada == i+x+1].iloc[0:12,0:7]))).T
        datos_b = datos_b.reset_index()
        corners_b = partidos[partidos.Jornada == i+x+1].iloc[0:12,-1].sum()#suma de los corners
        datos_b.insert(6,'corners', corners_b)

        jugadores_c = pd.DataFrame(np.concatenate(np.asarray(partidos[partidos.Jornada == i+x+2].iloc[0:12,0:1]))).T
        corners_c = partidos[partidos.Jornada == i+x+2].iloc[0:12,-1].sum()
        jugadores_c.insert(11,'corners', corners_c)

        total_v1 = pd.concat([datos_a,datos_b,jugadores_c],axis=1,ignore_index=True)
        total_v1 = total_v1.drop([79,86,87,88,89,91],axis=1)
        lista.append(total_v1)

    df_final = pd.concat(lista)
    df_final = df_final.reset_index()

    df_final = df_final.drop(0,axis=1)
    df_final = df_final.drop('index',axis=1)
    df_final.replace(np.nan,0)
    # df_final = df_final.replace(['local','visitante'],[0,1])
    return df_final

## Random Forest

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score
import itertools

#### 3- la función de dataframe vectorial se aplica por equipo:

In [8]:
df_final = Team('betis')

In [9]:
df_final[169] = df_final[169].replace((0,1,2,3),0)
df_final[169] = df_final[169].replace((4,5,6),1)
df_final[169] = df_final[169].replace((7,8,9,10,11,12,13),2)

In [16]:
clf = RandomForestClassifier()
X = df_final.iloc[:,:162]
y = df_final.iloc[:,162:163]

In [17]:
params={'criterion':['gini', 'entropy'],
        'max_depth': [2,4,8],# Maxima pofundidad del arbol
        'max_features': [2,3,5], # numero de features a considerar en cada split
        'max_leaf_nodes': [3,5,8], # maximo de nodos del arbol
        'min_impurity_decrease' : [0.02], # un nuevo nodo se hará si al hacerse se decrece la impureza
        'min_samples_split': [3], # The minimum number of samples required to split an internal node
        'n_estimators':[1, 2, 5, 10,20]
        }

#### 4- Aplicamos GridSearch para ver qué parámetros son los más adecuados:

In [19]:
X = np.array(X)
y = np.array(y)
from sklearn.model_selection import GridSearchCV
grid_solver = GridSearchCV(estimator = clf, # model to train
                   param_grid = params, # param_grid
                   scoring = make_scorer(f1_score, average ="macro"),
                   cv = 5)
model_result = grid_solver.fit(X,y)

/home/angmar/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/angmar/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/angmar/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/angmar/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expec

In [10]:
model_result.best_estimator_

NameError: name 'model_result' is not defined

In [22]:
model_result.best_params_

{'criterion': 'entropy',
 'max_depth': 4,
 'max_features': 5,
 'max_leaf_nodes': 8,
 'min_impurity_decrease': 0.02,
 'min_samples_split': 3,
 'n_estimators': 2}

In [ ]:
y = np.array(y)

#### 5- Aplicamos Feature Selection y Leave One Out al DataFrame para aplicar el algoritmo a un nuevo dataframe que reuna las columnas más relevantes del dataframe anterior:

In [58]:
def RFTeam(X):
    #df_final=X    
    df_final = Team(X)
    df_final = df_final.replace(np.nan, 0.0)
    df_final[169] = df_final[169].replace((0,1,2,3),0)
    df_final[169] = df_final[169].replace((4,5,6),1)
    df_final[169] = df_final[169].replace((7,8,9,10,11,12,13),2)
    X = df_final.iloc[:,:162]
    y = df_final.iloc[:,162:163]
    X = np.array(X)
    y = np.array(y)
    #y = y.replace(np.nan,0.0)
    y = y[:, 0]
    # Build a forest and compute the feature importances
    forest = RandomForestClassifier(n_estimators=100, criterion="entropy", max_depth=4, min_samples_split=3, max_features=2, max_leaf_nodes=8, min_impurity_decrease=0.02)
 #Change for R.Forest
    forest.fit(X, y)
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    #print("Feature ranking:")

    #for f in range(X.shape[1]):
      #  print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
    X = df_final.iloc[:,indices[:50]]
    X=X.replace(np.nan,0)
    X = np.array(X)

    loo = LeaveOneOut()
    loo.get_n_splits(X)
    yhat=[]
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        #CLASIFICADOR Random Forest
        clf = RandomForestClassifier(n_estimators=100, criterion="entropy", max_depth=4, min_samples_split=3, max_features=2, max_leaf_nodes=8, min_impurity_decrease=0.02)
        clf.fit(X_train, y_train)  ##X_train, y_train
        yhat1 = clf.predict(X_test)
        yhat.append(yhat1)       
        
    #print("Accuracy: ", accuracy_score(y,yhat))
    return accuracy_score(y,yhat)

#### 6- Hacemos la prueba con un equipo al azar:

In [57]:
RFTeam('betis')

(array([1., 1., 1., 0., 2., 0., 2., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
        2., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1.,
        2., 1.]),
 [array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([0.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([0.]),
  array([1.]),
  array([0.]),
  array([0.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([0.]),
  array([1.]),
  array([1.]),
  array([0.]),
  array([1.]),
  array([0.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([0.])])

#### 7- Mediante un bucle lo aplicamos a cada equipo y comparamos el nivel de precisión para cada uno:

In [19]:
L_Acc = []
for i in list_equip:
    L_Acc.append(RFTeam(i))
Dicc = dict(zip(list_equip,L_Acc))
Dicc

{'girona': 0.6666666666666666,
 'valladolid': 0.4166666666666667,
 'betis': 0.5,
 'levante': 0.3611111111111111,
 'celta': 0.5833333333333334,
 'espanyol': 0.6666666666666666,
 'villarreal': 0.4722222222222222,
 'real_sociedad': 0.5,
 'barcelona': 0.3333333333333333,
 'alaves': 0.5555555555555556,
 'eibar': 0.4166666666666667,
 'huesca': 0.4166666666666667,
 'rayo': 0.6111111111111112,
 'sevilla': 0.4722222222222222,
 'real_madrid': 0.3611111111111111,
 'getafe': 0.3611111111111111,
 'valencia': 0.4166666666666667,
 'atletico': 0.4722222222222222,
 'athletic': 0.6944444444444444,
 'leganes': 0.4444444444444444}

In [20]:
yhat

NameError: name 'yhat' is not defined

#### Tomamos como media de la precisión del algoritmo, la media del nivel de precisión para cada equipo. Cuando hagamos las predicciones para ver a qué partido apostar o no, deberemos de tener en cuenta los valores de predicción para cada equipo a la hora de tomar decisiones pues siempre tendremos más probabilidad de acierto con aquellos equipos cuya precisión sea del 0.66 que con aquellos que tengan un 0.4.
#### *Random Forest no nos da una precisión exacta, si corriésemos varias veces el código desde el principio, es resultado variaría entre el 0.48 y 0.52 de accuracy.

In [41]:
#Media Adaboost: 0.4375
#Media RForest: 0.49
#Media SVM: 0.44 (0.5 si obviamos los 3 ceros)- Sin Feature selection
#Media KNNt: 0.45 - Sin Feature Selection
Media_Random_Forest = sum(L_Acc)/20
Media_Random_Forest

0.49305555555555547

## Comparativa aciertos jornadas

In [79]:
def Comparador(X):
    #df_final=X    
    df_final = Team(X)
    df_final = df_final.replace(np.nan, 0.0)
    df_final[169] = df_final[169].replace((0,1,2,3),0)
    df_final[169] = df_final[169].replace((4,5,6),1)
    df_final[169] = df_final[169].replace((7,8,9,10,11,12,13),2)
    X = df_final.iloc[:,:162]
    y = df_final.iloc[:,162:163]
    X = np.array(X)
    y = np.array(y)
    #y = y.replace(np.nan,0.0)
    y = y[:, 0]
    # Build a forest and compute the feature importances
    forest = RandomForestClassifier(n_estimators=100, criterion="entropy", max_depth=4, min_samples_split=3, max_features=2, max_leaf_nodes=8, min_impurity_decrease=0.02)
 #Change for R.Forest
    forest.fit(X, y)
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    #print("Feature ranking:")

    #for f in range(X.shape[1]):
      #  print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
    X = df_final.iloc[:,indices[:50]]
    X=X.replace(np.nan,0)
    X = np.array(X)

    loo = LeaveOneOut()
    loo.get_n_splits(X)
    yhat=[]
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        #CLASIFICADOR Random Forest
        clf = RandomForestClassifier(n_estimators=100, criterion="entropy", max_depth=4, min_samples_split=3, max_features=2, max_leaf_nodes=8, min_impurity_decrease=0.02)
        clf.fit(X_train, y_train)  ##X_train, y_train
        yhat1 = clf.predict(X_test)
        yhat.append(yhat1)   
    y = list(y)
    Yhat = list(itertools.chain(*yhat))
        
    return y, Yhat

In [86]:
L_Aciertos = []
for j in list_equip: 
    Comparadore = Comparador(j)
    Acierto=[]
    for i in range(len(y)):
        if Comparadore[0][i]==Comparadore[1][i]:
            Acierto.append(1)
        else:
            Acierto.append(0)
    L_Aciertos.append(Acierto)

In [108]:
L_Aciertos

[[1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1],
 [0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1],
 [1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1],
 [0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1],
 [1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0],
 [0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,


In [88]:
Dicc_Ac = dict(zip(list_equip,L_Aciertos))
Dicc_Ac

{'girona': [1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1],
 'valladolid': [0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1],
 'betis': [1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1],
 'levante': [0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1],
 'celta': [1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0],
 'espanyol': [0,
  0,
  1,
  1,
  1,
  0,


In [122]:
Jornadas_Acc.iloc[:,:]=L_Aciertos

In [124]:
pd.set_option('display.max_columns', None)
Jornadas_Acc

,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
girona,1,1,1,0,0,1,0,0,1,1,1,1,1,1,0,1,0,1,1,0,0,1,1,1,0,1,1,1,1,0,1,0,1,1,0,1
valladolid,0,1,0,0,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,0,0,0,1
betis,1,1,1,0,0,0,0,1,1,1,0,1,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,1,1,0,1
levante,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,1,1
celta,1,1,0,1,1,0,0,0,1,0,0,1,1,0,0,0,1,1,1,1,0,1,1,0,0,1,0,0,0,1,0,1,0,0,0,0
espanyol,0,0,1,1,1,0,1,1,1,1,0,0,0,1,0,1,1,1,0,1,0,1,1,0,1,1,0,1,1,1,0,0,1,0,1,0
villarreal,0,0,1,0,0,1,0,0,1,1,0,0,1,1,1,0,0,0,1,0,1,0,0,1,0,1,0,1,1,0,0,0,0,0,1,1
real_sociedad,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,1,0,0,0,0,1,0,1,1,0,1,1,0,1,0,0,1,1,1,1,1
barcelona,0,0,0,0,1,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,1,1,0
alaves,1,1,0,0,1,0,1,0,1,0,1,1,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,0,1,0,0,0,0


#### El siguiente código nos muestra la cantidad de aciertos (2ª columna) en cada jornada (1ª columna) que predice el modelo, de la 3 a las 38.

In [127]:
Jornadas_Acc.sum(axis=None, skipna=None, level=None, numeric_only=None, min_count=0)  

3     10
4     10
5     10
6      5
7      9
8      9
9     10
10     7
11    12
12    12
13    10
14    10
15    10
16    11
17    11
18     7
19    10
20    10
21     8
22     7
23     7
24    10
25     8
26     8
27    11
28    15
29    12
30    10
31    13
32    11
33     8
34    11
35    10
36     7
37     9
38    12
dtype: int64

## Conclusión:
#### El programa no sirve para predecir todos los resultados, eso sería casi imposible en un juego donde hay demasiadas variables aleatorias y azarosas, está diseñado como herramienta de ayuda a aquellos que tengan conocimiento de fútbol y de la competición analizada en cuestión. Es especialmente útil para decectar partidos en los que habrá una cantidad muy alta de córners (normalmente, las apuestas con mejores cuotas).